In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime as dt
import pytz
import asyncio
import os
import json

#From binance-connector
from binance.spot import Spot

#From python-binance
from binance.client import Client

#import websockets

In [28]:

def filter_by_date(arr, dates):
  new_arr=[]

  for obj in arr:
    for date in dates:
      if date in obj["datetime"]:
        new_arr.append(obj)

  return new_arr


def get_market_cap(symbol, current_price, volume):
  return current_price * volume

def bubble_sort_by_attribute(arr, attr):
  length = len(arr) - 1
  sorted = False

  while not sorted:
    sorted = True
    for i in range(0, length):
      if arr[i][attr] > arr[i+1][attr]:
        sorted = False
        arr[i], arr[i+1] = arr[i+1], arr[i]

  return arr

def filter_repeated(arr):
  df = pd.DataFrame(arr)
  return df.drop_duplicates(subset=['symbol'], keep='first').to_dict('records')

def filter_repeated_reverse(arr):
  df = pd.DataFrame(arr)
  return df.drop_duplicates(subset=['symbol'], keep='last').to_dict('records')

def calculate_volatility(data, symbol):
  none = 0
  increment = 0
  decrement = 0

  for i in data:
    if symbol == i["symbol"]:
      if(i["open_price"] - i["close_price"] > 0):
        increment += 1

      if(i["open_price"] - i["close_price"] < 0):
        decrement += 1

  return abs(increment - decrement)


In [8]:
client = Client(api_key='ABzMGzpN1WChoBFHHxEaodEcBUlOgCaFcc75R8unXdFvgfcjNlPQlma5Eik0e4E0', api_secret='BYUbTAIVxtkzUAq5kq86xXWFv8Kw4ji2awFTjMrwVVP3R08eM6nkIrUmHUJCYOyU')
# fetch weekly klines since it listed

coins_arr = ["SOLUSDT", "BTCUSDT", "LINKUSDT", "DOGEUSDT", "NEOUSDT", "YFIUSDT", "ETHUSDT","BOMEUSDT", "PEPEUSDT", "LTOUSDT", "MKRUSDT", "PAXGUSDT", "SHIBUSDT", "DREPUSDT", "GNOUSDT", "AVAXUSDT"]

klines_arr = []

for coin in coins_arr:
  klines = client.get_historical_klines(coin, Client.KLINE_INTERVAL_1DAY, "1 Jan, 2015", "1 Jan 2024")
  info = client.get_symbol_info(coin)
  obj = {'klines': klines, 'info': info}
  klines_arr.append(obj)


In [29]:
data_arr = []

for obj in klines_arr:
  for kline in obj['klines']:
    symbol = obj['info']['symbol']
    timestamp = kline[0]
    datetime = dt.fromtimestamp(timestamp / 1000).strftime('%Y-%m-%d %H:%M:%S')
    open_price = float(kline[1]) # Open price
    high = float(kline[2]) # Highest price on opening
    low = float(kline[3]) # Lowest price on opening
    close_price = float(kline[4])  # Close price
    volume = float(kline[5]) #Volume
    number_of_trades = float(kline[8])
    range_high_low=high-low
    market_cap = get_market_cap(symbol, close_price, volume) # Close price is passed since it's the ending price
    coin_name = symbol  # Based on the symbol used

    data = {"coin_name": coin_name, "symbol": symbol,  "datetime": str(datetime),"open_price": open_price,'high': high, 'low':low, 'range': range_high_low,"close_price": close_price, "volume": volume, "number_of_trades": number_of_trades, "market_cap": market_cap}
    data_arr.append(data)



# Most Valuable Coins (MVC)
data_ending_2018 = filter_by_date(data_arr, ["2018-10", "2018-11", "2018-12"])
mvc_ending_2018 = bubble_sort_by_attribute(data_ending_2018, "market_cap")
mvc_ending_2018 = filter_repeated(mvc_ending_2018)

volatility_data_2018 = []
volatility_data_all_time = []

for i in coins_arr:
  obj = {
      "symbol": i,
      "volatility": calculate_volatility(data_arr, i)
  }

  volatility_data_all_time.append(obj)

for i in coins_arr:
  obj = {
      "symbol": i,
      "volatility": calculate_volatility(data_ending_2018, i)
  }

  volatility_data_all_time.append(obj)

# sorted_data_arr = bubble_sort_by_attribute(data_arr, "close_price")
sorted_data_arr = filter_repeated_reverse(data_arr)

value = 0
down = 0


over_the_average = []
on_average = []
under_the_average = []

for i in sorted_data_arr:
  value += i["close_price"]
  down+=1

average = value = value/down

for i in sorted_data_arr:

  if i["close_price"] > average:
    over_the_average.append(i)
  elif i["close_price"] < average:
    under_the_average.append(i)
  else:
    on_average.append(i)

volatility_data_all_time = bubble_sort_by_attribute(volatility_data_all_time, "volatility")
volatility_data_2018 = bubble_sort_by_attribute(volatility_data_2018, "volatility")

coins_all_time_file = open('./output/most_valuable_coins_all_time.json', 'w')
coins_all_time_file.write(json.dumps(data_arr, indent=2))

most_valuable_coins_ending_2018 = open('./output/most_valuable_coins_ending_2018.json', 'w')
most_valuable_coins_ending_2018.write(json.dumps(mvc_ending_2018, indent=2))

most_volatile_coins_all_time = open('./output/most_volatile_coins_all_time.json', 'w')
most_volatile_coins_all_time.write(json.dumps(volatility_data_all_time, indent=2))

most_volatile_coins_2018 = open('./output/most_volatile_coins_2018.json', 'w')
most_volatile_coins_2018.write(json.dumps(volatility_data_2018, indent=2))

ov_avg = open('./output/over_the_average_coins.json', 'w')
ov_avg.write(json.dumps(over_the_average, indent=2))

u_avg = open('./output/under_the_average_coins.json', 'w')
u_avg.write(json.dumps(under_the_average, indent=2))

o_avg = open('./output/average_coins.json', 'w')
o_avg.write(json.dumps(on_average, indent=2))

KeyboardInterrupt: 